In [1]:
! pip install requests
! pip install bs4   
! pip install pd


[notice] A new release of pip is available: 23.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import re
import os

# All State in One Code

Generating Links dynamically

In [3]:
base_url = "https://www.medindia.net/patients/hospital_search/hospitals-"
states = [
    #"chandigarh",
    #"chhattisgarh",
    #"dadra-and-nagar-haveli",
    #"daman-and-diu",
     #"delhi",
    #"puducherry",
     "punjab"
]


def generate_url(state, page):
    urls = []
    for p in range(1, page + 1):
        formatted_state = state.lower().replace(" ", "-")
        if p == 1:
            url = base_url + formatted_state + ".htm"
        else:
            url = base_url + formatted_state + "-" + str(p) + ".htm"
        urls.append(url)
    return urls


In [4]:
def scrape_hospital_data(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')

    hospital_divs = soup.find_all('h3', class_='vert-small-margin')
    hospital_data = []

    for hospital_div in hospital_divs:
        hospital_detail = hospital_div.find('a').text.strip()
        parts = [part.strip()
                 for part in hospital_detail.split(',') if part.strip()]

        name = parts[0] if len(parts) >= 1 else ''
        city = parts[1] if len(parts) >= 2 else ''
        state = parts[2] if len(parts) >= 3 else ''

        hospital_data.append({
            'Hospital Name': name,
            'City': city.title(),
            'State': state
        })

    df = pd.DataFrame(hospital_data)

    # Extract the filename from the URL
    fileName = re.search(r"\/hospitals-(.+?)(?:-\d+)?\.htm", url).group(1)

    if os.path.isfile(f'{fileName}.csv'):
        # Append data without headers
        df.to_csv(f'{fileName}.csv', mode='a', header=False, index=False)
    else:
        # Write data with headers
        df.to_csv(f'{fileName}.csv', mode='a', header=True, index=False)


for state in states:
    for links in generate_url(state, 141):
        scrape_hospital_data(links)
